# Usage of GC.Analysis.API for CPU Analysis 

In [ ]:
#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Newtonsoft.Json"
#r "nuget: YamlDotNet"
#r "nuget: Microsoft.Playwright, 1.16.0"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using XPlot.Plotly;

using System.IO;
using Newtonsoft.Json;

## Building and Using The GC Analysis API

In [ ]:
dotnet build .\GC.Analysis.API

In [ ]:
#r "./GC.Analysis.API/bin/Debug/net6.0/GC.Analysis.API.dll"

using GC.Analysis.API;

## Creating the Analyzer

In [ ]:
var SEGMENTS_TRACE_PATH = @"";
var REGIONS_TRACE_PATH  = @"";

In [ ]:
Analyzer segmentsAnalyzer = new Analyzer(tracePath: SEGMENTS_TRACE_PATH, processNames: new HashSet<string> { "" });

In [ ]:
Analyzer regionsAnalyzer = new Analyzer(tracePath: REGIONS_TRACE_PATH, processNames: new HashSet<string> { "" });

## Summarization

In [ ]:
segmentsAnalyzer.SummarizeTrace(processName: "")

In [ ]:
regionsAnalyzer.SummarizeTrace(processName: "")

In [ ]:
segmentsAnalyzer.GetProcessGCData("").Single().Compare(new[] { regionsAnalyzer.GetProcessGCData("").Single() })

## CPU Analysis

### Creating the CPU Analysis

In [ ]:
segmentsAnalyzer.AddCPUAnalysis()

In [ ]:
string segmentsSerialized = JsonConvert.SerializeObject(segmentsAnalyzer.CPUAnalyzer.GetCPUDataForProcessName("").First());
CPUProcessData segmentsCPUData = segmentsAnalyzer.CPUAnalyzer.GetCPUDataForProcessName("").Single();

In [ ]:
regionsAnalyzer.AddCPUAnalysis(symbolPath: @"");
CPUProcessData regionsCPUData = regionsAnalyzer.CPUAnalyzer.GetCPUDataForProcessName("").Single();

### Summarization

#### Major GC Phase Summarization

In [ ]:
segmentsCPUData.GetPerPhaseSummary()

In [ ]:
regionsCPUData.GetPerPhaseSummary()

In [ ]:
segmentsCPUData.Compare(new [] { regionsCPUData })

#### Summarization By GC Type and Generation

In [ ]:
segmentsCPUData.GetPerGenerationSummary() // TODO: Check where the background phases went. BGC count - attribution.

#### Aux Methods

In [ ]:
segmentsCPUData.GetPerGCMethodCost("gc_heap::mark_phase").Select(gc => ( gc.GC.Number, gc.Count ))

In [ ]:
segmentsCPUData.GetPerGCMethodCost(methodName: "gc_heap::mark_through_cards_for_segments", 
                                   caller: "gc_heap::relocate_phase").Select(gc => (gc.GC.Number, gc.Count))

### Charting

#### Charting Counts For a Single GC Method

##### Charting Counts For a Single GC Method Using A Sorted Dictionary 

In [ ]:
(string, List<CPUInfo>) data = 
    ("gc_heap::mark_phase", segmentsCPUData.GetPerGCMethodCost("gc_heap::mark_phase"));

CPUCharting.ChartCountForGCMethod(data, "Mark Phase Data")

##### Charting Counts For a Single Method GC Using CPU Process Data

In [ ]:
// With Inclusive Count.
segmentsCPUData.ChartCountForGCMethod(methodName: "gc_heap::plan_phase", title: "Plan Phase")

In [ ]:
// With Exclusive Count.
segmentsCPUData.ChartCountForGCMethod(methodName: "gc_heap::plan_phase", title: "Plan Phase - Exclusive Count", isInclusiveCount: false)

### Charting Counts For a Single GC Method With Caller

In [ ]:
segmentsCPUData.ChartCountForGCMethod(methodName: "gc_heap::mark_through_cards_for_segments", 
                                      title: "Inc. cost of mark_through_cards_for_segments for the Mark Phase", 
                                      caller: "gc_heap::mark_phase")

In [ ]:
segmentsCPUData.ChartCountForGCMethod(methodName: "gc_heap::mark_through_cards_for_segments", 
                                      title: "Inc. cost of mark_through_cards_for_segments for the Relocate Phase", 
                                      caller: "gc_heap::relocate_phase")

#### Charting Counts For Multiple GC Methods

##### Charting Counts For Multiple GC Methods Using A List

In [ ]:
(string, List<CPUInfo>) markPhaseData = 
    ("gc_heap::mark_phase", segmentsCPUData.GetPerGCMethodCost("gc_heap::mark_phase"));

(string, List<CPUInfo>) planPhaseData = 
    ("gc_heap::plan_phase", segmentsCPUData.GetPerGCMethodCost("gc_heap::plan_phase"));

(string, List<CPUInfo>) relocate_phase = 
    ("gc_heap::relocate_phase", segmentsCPUData.GetPerGCMethodCost("gc_heap::relocate_phase"));

CPUCharting.ChartCountForGCMethods(new [] { markPhaseData, planPhaseData, relocate_phase }, "Custom Phase Data")

In [ ]:
// TODO: Add Filter Example instead of GCs.

(string, List<CPUInfo>) markPhaseData = 
    ("gc_heap::mark_phase", segmentsCPUData.GetPerGCMethodCost(methodName: "gc_heap::mark_phase", gcsToConsider: new HashSet<int>{ 196, 200 }));

(string, List<CPUInfo>) planPhaseData = 
    ("gc_heap::plan_phase", segmentsCPUData.GetPerGCMethodCost(methodName: "gc_heap::plan_phase", gcsToConsider: new HashSet<int> { 196, 200 }) );

CPUCharting.ChartCountForGCMethods(new [] { markPhaseData, planPhaseData }, "Custom Phase Data")

##### Charting Counts For The Same Phase Between For Different Traces

In [ ]:
(string, List<CPUInfo>) markPhaseData = 
    ("gc_heap::mark_phase - Segments", segmentsCPUData.GetPerGCMethodCost(methodName: "gc_heap::mark_phase"));

(string, List<CPUInfo>) planPhaseData = 
    ("gc_heap::plan_phase - Regions", regionsCPUData.GetPerGCMethodCost(methodName: "gc_heap::mark_phase"));

CPUCharting.ChartCountForGCMethods(new [] { markPhaseData, planPhaseData }, "Mark Phase: Segments vs. Regions")

##### Charting Counts For Multiple GC Methods Using CPU Process Data

In [ ]:
segmentsCPUData.ChartCountForGCMethods(methodNames: new [] { "gc_heap::mark_phase", "gc_heap::plan_phase", "gc_heap::relocate_phase", "gc_heap::compact_phase" }, 
                                 title: "Major GC Phases")

#### Charting GC Data With CPU Data

In [ ]:
GCProcessData processData = segmentsAnalyzer.GetProcessGCData("").Single();

In [ ]:
(string, List<CPUInfo>) cpuData = ("Mark Phase", segmentsCPUData.GetPerGCMethodCost("gc_heap::mark_phase"));
(string, List<double>) gcData  = ("Msec", processData.GCs.Select(gc => gc.PauseDurationMSec));
CPUCharting.ChartCountForGCMethodWithGCData(cpuData, gcData, "Mark Phase Inc. Count vs. Pause Duration (MSec)")

### Generic Charting

In [ ]:
(string, List<CPUInfo>) markPhaseData = 
    ("gc_heap::mark_phase", segmentsCPUData.GetPerGCMethodCost(methodName: "gc_heap::mark_phase"));

bool Gen0Filter(CPUInfo cpuInfo)
{
    return cpuInfo.GC.Generation == 0;
}

bool Gen0Filter(TraceGC gc)
{
    return gc.Generation == 0;
}

List<CPUInfo> filteredGen0CPUData = markPhaseData.Item2.Where(Gen0Filter);
List<TraceGC> traceGC = processData.GCs.Where(Gen0Filter);

GCCharting.ChartGCData("Mark Phase Inc. Count Per GC", 
                       new AxisInfo 
                       { 
                           Name = "Mark Phase Inc", 
                           XAxis = traceGC.Select(gc => (double)gc.Number), 
                           YAxis = filteredGen0CPUData.Select(gc => (double)gc.Count) 
                       })

### Mark Phase Analysis

#### Chart Per Generation and Type 

In [ ]:
processData.ChartStatisticsOfMarkPhaseByType(generation: 0, type: MarkRootType.MarkStack)

#### Chart All Mark Root Types For a Generation 

In [ ]:
processData.ChartAverageMarkPhaseTimeByMarkType(generation: 0, types: new [] { MarkRootType.MarkStack, MarkRootType.MarkHandles, MarkRootType.MarkOlder })

## Source Code Analysis

In [ ]:
var BASE_PATH = @"";
Analyzer analyzer = new Analyzer(tracePath: BASE_PATH, processNames: new HashSet<string> { "" });

In [ ]:
analyzer.AddCPUAnalysis(symbolPath : BASE_PATH);

In [ ]:
CPUProcessData cpuAnalyzer = analyzer.CPUAnalyzer.GetCPUDataForProcessName("").Single();

In [ ]:
cpuAnalyzer.SetSourcePath(BASE_PATH);
cpuAnalyzer.Annotate("gc_heap::garbage_collect")

## Affinitized CPU Analysis

In [ ]:
var gcData = analyzer.GetProcessGCData("").Single();

### Summary

In [ ]:
Dictionary<string, AffinitizedCPUData> get_stats_gcs_with_large_diff(List<TraceGC> gcs, double threshold)
{
    // Per process name affinitized cpu data.
    Dictionary<string, AffinitizedCPUData> data = new();

    for(int i = 0; i < gcs.Count; i++)
    {
        double diff_between_suspend_ee_stop_and_gc_start = gcs[i].StartRelativeMSec - (gcs[i].PauseStartRelativeMSec + gcs[i].SuspendDurationMSec);
        if (diff_between_suspend_ee_stop_and_gc_start > threshold)
        {
            // Processor -> < ProcessID, AffinitizedCPUData >
            Dictionary<int, Dictionary<int, AffinitizedCPUData>> affinitizedAnalysis = gcData.GetAffinitizedAnalysis(gcs[i].PauseStartRelativeMSec + gcs[i].SuspendDurationMSec, gcs[i].StartRelativeMSec);
            foreach(var processor in affinitizedAnalysis)
            {
                foreach(var process in processor.Value)
                {
                    if (!data.TryGetValue(process.Value.Name, out var affinitizedCPUData))
                    {
                        data[process.Value.Name] = affinitizedCPUData = new AffinitizedCPUData
                        {
                            Name = process.Value.Name,
                            ProcessorNumber = -1, // All processors
                        };
                    }

                    affinitizedCPUData.NumberOfSamples += process.Value.NumberOfSamples;
                }
            }
        }
    }

    return data;
}

Dictionary<string, AffinitizedCPUData> stats = get_stats_gcs_with_large_diff(gcData.GCs, 8);

#### Number of Samples

In [ ]:
float total_number_of_samples_from_non_gc_threads = 0;
float total_number_of_samples_from_gc_threads = 0;

foreach(var process_data in stats)
{
    if (process_data.Key != "GC Thread")
    {
        total_number_of_samples_from_non_gc_threads += process_data.Value.NumberOfSamples;
    }

    else
    {
        total_number_of_samples_from_gc_threads += process_data.Value.NumberOfSamples;
    }
}

Console.WriteLine($"Number of Samples from Threads >= GC Thread Priority of 14: {total_number_of_samples_from_non_gc_threads}");
Console.WriteLine($"Number of Samples from GC Threads : {total_number_of_samples_from_gc_threads}");

#### Per Process Summarization

In [ ]:
// Get total samples

float total_samples = 0;
foreach (var process_data in stats)
{
    total_samples += process_data.Value.NumberOfSamples;
}

StringDataFrameColumn process_name = new("Process Name");
DoubleDataFrameColumn number_of_samples = new("Number of Samples");
DoubleDataFrameColumn percentage_of_samples = new("% of Samples");

foreach (var process_data in stats)
{
    process_name.Append(process_data.Key);
    number_of_samples.Append(process_data.Value.NumberOfSamples);
    percentage_of_samples.Append(Math.Round(process_data.Value.NumberOfSamples / total_samples * 100, 2));
    Console.WriteLine($"For: {process_data.Key}, Number of Samples: {process_data.Value.NumberOfSamples}, % of Samples: {Math.Round((process_data.Value.NumberOfSamples / total_samples) * 100, 2)}%");
}

var df = new DataFrame(process_name, number_of_samples, percentage_of_samples);

#### Save Results To Markdown 

In [ ]:
df.Display();
df.ToMarkdown("./AffinitizedResults.md");

## Debugging

In [ ]:
Console.WriteLine($"Current Process ID: {System.Diagnostics.Process.GetCurrentProcess().Id}");

#!about